In [7]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers

In [8]:
# one-hot encode braid word and pad to consistent length
def encode_braid(braid, maxStrands, maxCrossings):

    b = tf.keras.utils.to_categorical(braid, num_classes=maxStrands*2)

    for i in range(maxCrossings - len(b)):
            b = np.append(b, [np.zeros(maxStrands*2)], axis=0)

    return b

In [9]:
# turn list of lists of equivalent braids with corresponding answers into trainable data
def parse_equivalent_braids(allBraids, answers, maxStrands, maxCrossings):

    x_data = []
    y_data = []

    # zip list of equivalent braids with same label
    for equivalentBraids, answer in zip(allBraids, answers):
        for braid in equivalentBraids:
            x_data.append(braid)
            y_data.append(answer)

    # encode each braid word
    x_data = [encode_braid(x, maxStrands, maxCrossings) for x in x_data]

    return np.asarray(x_data), np.asarray(y_data)

In [10]:
# read pre-parsed knot data with braid words and equivalent braids

knot_data = pd.read_csv('../data/knot_data_jones.csv')
knot_data["Equivalent Braids"] = knot_data["Equivalent Braids"].apply(eval)
knot_data["Jones"] = knot_data["Jones"].apply(eval)

In [11]:
eq = knot_data["Equivalent Braids"]

maxCrossings = max([max([len(braid) for braid in braids]) for braids in eq])
maxStrands = max([max([max([abs(b) for b in braid]) for braid in braids]) for braids in eq])

minPower = -13
maxPower = 16
nCoefficients = maxPower - minPower + 1

In [12]:
# split dataset for training

train_split = .8

train_data = knot_data.sample(frac=train_split)
test_data = knot_data.drop(train_data.index)

x_train, y_train = parse_equivalent_braids(train_data["Equivalent Braids"],
                    train_data["Jones"], maxStrands, maxCrossings)

x_test, y_test = parse_equivalent_braids(test_data["Equivalent Braids"],
                    test_data["Jones"], maxStrands, maxCrossings)

In [13]:
# build model from functional api

inputs = keras.Input(shape=(maxCrossings, maxStrands*2))

# recurrent version
x = layers.Masking()(inputs)
x = layers.LSTM(128)(x)

# linear version
#x = layers.Flatten()(inputs)
#x = layers.Dense(64, activation="relu")(x)
#x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(nCoefficients)(x)

In [14]:
model = keras.Model(inputs=inputs, outputs=outputs, 
    name="jones_polynomial")

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_squared_error"])

model.summary()

Model: "jones_polynomial"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 12)]          0         
                                                                 
 masking (Masking)           (None, 19, 12)            0         
                                                                 
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 dense (Dense)               (None, 30)                3870      
                                                                 
Total params: 76,062
Trainable params: 76,062
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=10
)

Epoch 1/10


2023-05-04 13:29:30.835905: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-04 13:29:32.262273: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_23'


595/595 [==============================] - 17s 22ms/step - loss: 74.9083 - mean_squared_error: 74.9083
Epoch 2/10
175/595 [=======>......................] - ETA: 8s - loss: 69.9782 - mean_squared_error: 69.9782

KeyboardInterrupt: 

In [58]:
model.evaluate(x_test, y_test)

594/594 [==============================] - 2s 4ms/step - loss: 46.3204 - mean_squared_error: 46.3204


[46.320430755615234, 46.320430755615234]

In [40]:
def predict_coefficients(braid, maxStrands, maxCrossings):
    return np.asarray(model(np.array([encode_braid(braid, maxStrands, maxCrossings)])))[0]

In [41]:
braid = test_data.iloc[10]
word = braid["Equivalent Braids"][0]
jones = braid["Jones"]

prediction = predict_coefficients(word, maxStrands, maxCrossings)

In [42]:
for i in range(len(jones)):
    print(jones[i], round(prediction[i]), round(prediction[i] - jones[i]))

0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 -1 -1
1 1 0
-2 -3 -1
3 4 1
-5 -6 -1
6 7 1
-6 -8 -2
6 7 1
-4 -5 -1
3 4 1
-1 -2 -1
0 1 1
0 0 0
0 0 0
0 0 0
0 -1 -1
0 1 1
0 -1 -1
0 1 1
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
